# COMP5318 - Machine Learning and Data Mining 

## Deep Residual Network

#### Not for marking purpose

### 1 Resnet (transfer learning)

In [75]:
import torch
print(torch.__version__)

0.4.1


In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torchvision.transforms as transforms

#X_tensor = torch.from_numpy(X_train)
#trainloader = torch.utils.data.TensorDataset()

In [41]:
np.mean(X_train/255)

0.2860405969887955

In [42]:
np.std(X_train/255)

0.35302424451492237

In [53]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.286], [0.353])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.286], [0.353])
    ]),
}

trainset = torchvision.datasets.FashionMNIST(root='./Fashion MNIST', train=True,
                                        download=True, transform=data_transforms['train'])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

valset = torchvision.datasets.FashionMNIST(root='./Fashion MNIST', train=False,
                                       download=True, transform=data_transforms['val'])
valloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot')

In [71]:
image_datasets = {x: datasets.FashionMNIST(root='./Fashion MNIST', train=True,
                                        download=True, transform=data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [72]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [73]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [74]:
model_ft = models.resnet18(pretrained=True)
model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 18)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=8)

Epoch 1/8
----------
train Loss: 0.7573 Acc: 0.7320
val Loss: 0.3423 Acc: 0.8737

Epoch 2/8
----------
train Loss: 0.5495 Acc: 0.8013
val Loss: 0.2625 Acc: 0.9032

Epoch 3/8
----------


KeyboardInterrupt: 

### 2 Resnet Results

#### From the results above, we can see the codes are right and accuracy is pretty high, but due to the hardware limitation, we cancelled the training process and used other's GPU to run and the results are shown below:

In [77]:
# GPU: GTX1080
# Training complete in 63m 33s
# Best val Acc: 0.950517

# Epoch 0/24
# ----------
# train Loss: 0.7555 Acc: 0.7302
# val Loss: 0.3314 Acc: 0.8803

# Epoch 1/24
# ----------
# train Loss: 0.5473 Acc: 0.8049
# val Loss: 0.2661 Acc: 0.9038

# Epoch 2/24
# ----------
# train Loss: 0.4974 Acc: 0.8194
# val Loss: 0.2311 Acc: 0.9151

# Epoch 3/24
# ----------
# train Loss: 0.4669 Acc: 0.8313
# val Loss: 0.2031 Acc: 0.9257

# Epoch 4/24
# ----------
# train Loss: 0.4374 Acc: 0.8426
# val Loss: 0.2044 Acc: 0.9257

# Epoch 5/24
# ----------
# train Loss: 0.4262 Acc: 0.8452
# val Loss: 0.1847 Acc: 0.9326

# Epoch 6/24
# ----------
# train Loss: 0.4115 Acc: 0.8500
# val Loss: 0.1794 Acc: 0.9345

# Epoch 7/24
# ----------
# train Loss: 0.3768 Acc: 0.8619
# val Loss: 0.1524 Acc: 0.9446

# Epoch 8/24
# ----------
# train Loss: 0.3655 Acc: 0.8668
# val Loss: 0.1505 Acc: 0.9452

# Epoch 9/24
# ----------
# train Loss: 0.3565 Acc: 0.8713
# val Loss: 0.1478 Acc: 0.9466

# Epoch 10/24
# ----------
# train Loss: 0.3545 Acc: 0.8708
# val Loss: 0.1440 Acc: 0.9482

# Epoch 11/24
# ----------
# train Loss: 0.3550 Acc: 0.8702
# val Loss: 0.1446 Acc: 0.9479

# Epoch 12/24
# ----------
# train Loss: 0.3525 Acc: 0.8716
# val Loss: 0.1409 Acc: 0.9491

# Epoch 13/24
# ----------
# train Loss: 0.3493 Acc: 0.8732
# val Loss: 0.1442 Acc: 0.9477

# Epoch 14/24
# ----------
# train Loss: 0.3444 Acc: 0.8750
# val Loss: 0.1406 Acc: 0.9492

# Epoch 15/24
# ----------
# train Loss: 0.3416 Acc: 0.8763
# val Loss: 0.1398 Acc: 0.9500

# Epoch 16/24
# ----------
# train Loss: 0.3413 Acc: 0.8780
# val Loss: 0.1393 Acc: 0.9497

# Epoch 17/24
# ----------
# train Loss: 0.3453 Acc: 0.8740
# val Loss: 0.1397 Acc: 0.9497

# Epoch 18/24
# ----------
# train Loss: 0.3415 Acc: 0.8766
# val Loss: 0.1391 Acc: 0.9494

# Epoch 19/24
# ----------
# train Loss: 0.3417 Acc: 0.8754
# val Loss: 0.1392 Acc: 0.9498

# Epoch 20/24
# ----------
# train Loss: 0.3427 Acc: 0.8759
# val Loss: 0.1377 Acc: 0.9503

# Epoch 21/24
# ----------
# train Loss: 0.3438 Acc: 0.8762
# val Loss: 0.1375 Acc: 0.9505

# Epoch 22/24
# ----------
# train Loss: 0.3432 Acc: 0.8763
# val Loss: 0.1378 Acc: 0.9501

# Epoch 23/24
# ----------
# train Loss: 0.3436 Acc: 0.8756
# val Loss: 0.1379 Acc: 0.9505

# Epoch 24/24
# ----------
# train Loss: 0.3406 Acc: 0.8774
# val Loss: 0.1384 Acc: 0.9498

# Training complete in 63m 33s
# Best val Acc: 0.950517